# mIoU Analysis with Visualization
Analyze and visualize mIoU performance metrics with error bands using seaborn lineplot.

## Section 1: Import Required Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

## Section 2: Load Data with Pandas

In [ ]:
# Load the data from CSV
df = pd.read_csv('Sparsity_Cityscapes_split.csv')

# Display first few rows
print("First few rows of the data:")
print(df.head(10))
print("\nData shape:", df.shape)
print("\nColumn names:", df.columns.tolist())

## Section 3: Explore and Prepare Data

In [ ]:
# Check for missing values and data types
print("Data Info:")
print(df.info())
print("\nMissing values:")
print(df.isnull().sum())
print("\nData types:")
print(df.dtypes)

# Display basic statistics
print("\nBasic Statistics:")
print(df[['mIoU', 'mIoU_var']].describe())

In [ ]:
# Filter data - remove the baseline row with no preprocessing for clearer visualization
df_filtered = df[df['Preprocessing'] != '-'].copy()

# Create a combined sparsity label for better readability
df_filtered['Sparsity_Label'] = df_filtered['Sparsity'].fillna('No Sparsity')
df_filtered['Sparsity_Label'] = df_filtered['Sparsity_Label'].replace('-', 'No Sparsity')

print(f"Original data shape: {df.shape}")
print(f"Filtered data shape: {df_filtered.shape}")
print(f"\nUnique preprocessing methods: {df_filtered['Preprocessing'].unique()}")
print(f"Unique sparsity levels: {df_filtered['Sparsity_Label'].unique()}")

## Section 4: Prepare Data for Visualization with Error Bands

In [ ]:
# Calculate confidence bounds for error bands
df_filtered['mIoU_lower'] = df_filtered['mIoU'] - df_filtered['mIoU_var']
df_filtered['mIoU_upper'] = df_filtered['mIoU'] + df_filtered['mIoU_var']

# Sort by preprocessing and sparsity for better visualization
sparsity_order = ['No Sparsity', 'percent_90%', 'percent_80%', 'percent_70%', 'percent_60%', 
                  'percent_50%', 'percent_40%', 'threshold_0.02', 'threshold_0.01', 
                  'threshold_0.0075', 'threshold_0.005']

# Create a numeric index for sparsity levels for x-axis
df_filtered['sparsity_numeric'] = df_filtered['Sparsity_Label'].apply(
    lambda x: sparsity_order.index(x) if x in sparsity_order else -1
)

print("Data prepared for visualization:")
print(df_filtered[['Preprocessing', 'Sparsity_Label', 'mIoU', 'mIoU_var', 'mIoU_lower', 'mIoU_upper']].head(10))

## Section 5: Create Lineplot with Error Bands

In [ ]:
# Create the lineplot with error bands
fig, ax = plt.subplots(figsize=(16, 8))

# Plot lines and markers for each preprocessing method
for preprocessing in df_filtered['Preprocessing'].unique():
    data_subset = df_filtered[df_filtered['Preprocessing'] == preprocessing].sort_values('sparsity_numeric')
    
    # Plot main line
    ax.plot(data_subset['sparsity_numeric'], data_subset['mIoU'], 
            marker='o', label=preprocessing, linewidth=2.5, markersize=8)
    
    # Add error bands
    ax.fill_between(data_subset['sparsity_numeric'], 
                     data_subset['mIoU_lower'], 
                     data_subset['mIoU_upper'], 
                     alpha=0.15)

# Customize the plot
ax.set_xlabel('Sparsity Level', fontsize=12, fontweight='bold')
ax.set_ylabel('mIoU Score', fontsize=12, fontweight='bold')
ax.set_title('mIoU Performance by Preprocessing Method and Sparsity Level\n(Error Bands show ± Standard Deviation)', 
             fontsize=14, fontweight='bold', pad=20)

# Set x-axis labels
ax.set_xticks(range(len(sparsity_order)))
ax.set_xticklabels(sparsity_order, rotation=45, ha='right')

ax.legend(fontsize=11, loc='best', framealpha=0.9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Visualization complete!")

In [ ]:
# Alternative: Seaborn lineplot with custom error handling
fig, ax = plt.subplots(figsize=(16, 8))

# Use seaborn lineplot with hue for preprocessing methods
sns.lineplot(data=df_filtered, x='sparsity_numeric', y='mIoU', 
             hue='Preprocessing', marker='o', markersize=8, 
             linewidth=2.5, ax=ax, palette='husl')

# Add error bands manually for each preprocessing method
for preprocessing in df_filtered['Preprocessing'].unique():
    data_subset = df_filtered[df_filtered['Preprocessing'] == preprocessing].sort_values('sparsity_numeric')
    ax.fill_between(data_subset['sparsity_numeric'], 
                     data_subset['mIoU_lower'], 
                     data_subset['mIoU_upper'], 
                     alpha=0.1)

# Customize
ax.set_xlabel('Sparsity Level', fontsize=12, fontweight='bold')
ax.set_ylabel('mIoU Score', fontsize=12, fontweight='bold')
ax.set_title('mIoU Performance with Error Bands (Seaborn Lineplot)', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(range(len(sparsity_order)))
ax.set_xticklabels(sparsity_order, rotation=45, ha='right')
ax.legend(title='Preprocessing Method', fontsize=10, title_fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()